# <center>Engenharia de Conhecimento 21/22</center>
### <center>Mini-Projecto 2 </center>
## <center><font color='green'>I am Casper, the friendly ghost, but where am I?</font></center> 

<img src="files/imagens/casper.gif" alt="Drawing" style="width: 200px;"/> 

## Introdução
Este projecto é sobre o problema da localização. Na nossa história que começou no primeiro projecto, temos um fantasma que partindo em geral de uma posição desconhecida, se movimenta empurrado pelo vento durante um período discreto de tempo, num espaço com 1 ou 2 dimensões, também discreto. A acção do vento é também incerta mas a sua dinâmica probabilística é conhecida.

Neste projecto, iremos aplicar as Redes de Bayes ao mesmo cenário do projecto 1, e também a uma variante um pouco mais complexa:

    1. Modelizando o cenário do projecto 1 como uma Rede de Bayes, que tem limitações devido à falta de escalabilidade do processo de inferência a que chamámos de Naive. Poderão comprová-lo facilmente ao aumentarem o número de células no espaço e/ou a duração da acção do vento. 

    2. Para contornar a falta de eficiência, iremos tirar partido de existir um padrão que se repete nas Redes de Bayes de 1. Assim,iremos construir uma Rede de Bayes mínima formada apenas por 2 nós (representando a posição corrente e a posição seguinte), que será utilizada de um modo repetido, executando a inferência instante a instante, i.e. calculando a crença do fantasma na sua posição no instante seguinte, partindo da crença na sua posição no instante corrente. Este cálculo será repetido tantas vezes quantos os instantes que formam o período de tempo do cenário.

    3. Aumentaremos a complexidade do problema, embora reduzamos o espaço a uma só dimensão em forma de donut. Neste cenário mais complexo, o fantasma consegue percepcionar um conjunto de marcos (landmarks) associados a cada célula do espaço. Mas há complicações: mais uma vez a incerteza predomina e os sensores são ruidosos, e nem sempre identificam bem os marcos das células. Assim, o objectivo é que o fantasma consiga localizar-se modelizando o problema através de uma Rede de Bayes e calculando a probabilidades condicionais da sua posição dada uma sequência de valores dos sensores que são utilizados com um período variável.

Cada um destes casos corresponderá ao desenvolvimento de o, sendo as duas primeiras cotadas para 7 valores cada uma e a terceira para 6 valores. Todas as funções serão corrigidas automaticamente através de um conjunto de testes, alguns visíveis, outros no domínio do invisibili.

## O Problema da Localização

<img src="files/imagens/Particle_filters.gif" alt="Drawing" style="width: 400px;"/> 

Para vos apresentar o problema da localização nada melhor do que o nosso convidado [Sebastien Thrun](https://pt.wikipedia.org/wiki/Sebastian_Thrun), ligado ao automóvel inteligente da Google e à plataforma de ensino [Udacity](https://www.udacity.com/). Notem que o cenário que ele apresenta não é completamente idêntico ao deste projecto, mas serve como introdução ao problema da localização em que temos um espaço 1D, formado por células discretas, em forma de donut e em que associado a cada célula podemos ter paredes ou portas.

In [1]:
from IPython.display import Audio

In [2]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/n1EacrqyCs8?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>

## Poção Mágica
Como sabem, está comprovado que a melodia assobiada por [Ken Barrie](https://en.wikipedia.org/wiki/Ken_Barrie), ou melhor por Leslie Hulme, é um forte estímulo ao mundo Bayseano. É uma poção mágica com super-poderes ainda não completamente conhecidos, à qual podem recorrer quando acharem que estão a precisar de um reforço, mas não convém abusarem.

<img src="files/imagens/asterix.gif" alt="Drawing" style="width: 300px;"/> 

In [3]:
%%HTML
<audio controls>
  <source src=".\imagens\MyFamilyOtherAnimals1987ThemeSong.mp3">
</audio>

<img src="files/imagens/cadeiaFantasma.PNG" alt="Drawing" style="width: 700px;"/> 

### Função 1: *fantasma_RB*
Considerem exactamente o mesmo cenário do mini-projecto 1, em que temos um fantasma com uma posição inicial incerta e com uma dinâmica de movimento devido ao vento também incerta num espaço de 1 ou 2 dimensões. Na nossa modelização, a distribuição de probabilidade inicial é definida pela função `init_dist` e a dinâmica do movimento é expressa através da função `go`. O tempo flui de forma discreta, e o vento exerce a sua influência em cada tic do relógio, sendo a posição do fantasma no instante seguinte dependente apenas da sua posição no instante corrente. Cada posição do fantasma no instante $t_i$ é dada pela variável aleatória $X_{i}$, sendo o instante inicial $t_{0}$. Assim, tomando em consideração que $X_i$ é condicionalmente independente de qualquer $X_j (j >= 0, j < i-1)$ dado $X_{i-1}$, modelizem o problema através de uma Rede de Bayes, considerando um número máximo de instantes $t_{Max}$.

O objectivo é construírem uma função `fantasma_RB(distIni, max, movimentos, donut=False)`, que recebe como argumentos a distribuição inicial, o instante máximo da simulação e o modelo de movimento (representado nos dois argumentos movimentos e donut), e devolve uma instância da classe `RedeBayes`. Esta Rede de Bayes representa a distribuição conjunta:

$$P(X_0,X_1,X_2,...,X_{Max})=P(X_0)P(X_1|X_0)P(X_2|X_1)...P(X_{Max}|X_{Max-1})$$

**Nota:** para testar esta função, terão de fazer uso das funções `initDist` e `go`, que estão no ficheiro anexo *funcoes_fantasma.py* e que já conhecem, tendo sido encomendadas no mini-projecto I.

**Importante:** Durante as aulas práticas tiveram apenas contacto com a implementação em Python de Redes de Bayes Booleanas. O bloco de notas *RedeBayesGenericas.ipynb* apresenta as novas classes Python (`NoBayes` e `RedeBayes`) que implementam uma Rede de Bayes com varíáveis não restringidas a domínios booleanos. Essas classes constam também em *probabilityPlus.py*. 

**Atenção:** Para que não haja conflito entre a representação bidimensional do espaço e a forma como se representa as tabelas de probabilidades condicionais na classe `RedeBayes`, é necessário representar as células do espaço como `strings`. Por exemplo, a célula $1$ será $'1'$ e a célula $(3,3)$ será $'(3,3)'$.

Não têm de se preocupar com a programação defensiva. Os testes cumprem sempre com os inputs pedidos.

### Testes da Função 1

#### Teste 1

```python
ini = initDist(5,[1])
f=fantasma_RB(ini,5,['E','.'])
print(f.variables)

Output:
['X0', 'X1', 'X2', 'X3', 'X4', 'X5']
```

#### Teste 2

```python
ini = initDist(5,[1])
f=fantasma_RB(ini,5,['E','.'])
print(f.variable_values('X0'))

Output:
['1', '2', '3', '4', '5']
```

#### Teste 3

```python
ini = initDist(5,[1])
f=fantasma_RB(ini,5,['E','.'])
printSorted(f.variable_node('X0').cpt[()].prob)

Output:
{'1': 1.0, '2': 0, '3': 0, '4': 0, '5': 0}
```

#### Teste 4

```python
ini = initDist(5,[1])
f=fantasma_RB(ini,5,['E','.'])
display_cpt(f.variable_node('X0'))

Output:
P(X0|):
+----+------+
| X0 | Prob |
+----+------+
| 1  | 1.0  |
| 2  |  0   |
| 3  |  0   |
| 4  |  0   |
| 5  |  0   |
+----+------+
```

<img src="files/imagens/RedeBayesPadraoFantasma.PNG" alt="Drawing" style="width: 400px;"/> 


### Função 2: *fantasma_repeat*
Notem que ao utilizar uma rede de Bayes para representar a distribuição conjunta envolvendo $X_{0}$, $X_{1}$...$X_{i}$ poupamos no espaço quando comparamos com o armazenamento explícito numa tabela. No entanto, a inferência não escala como poderão facilmente comprovar ao aumentarem $t_{Max}$ ou mesmo o número de células do espaço 1D ou 2D.

Considerando que apenas queremos inferir a probabilidade marginal de $X_i$, isto é a posição do fantasma no intante $t_i$, partindo da distribuição inicial, $X_0$, no instante $t_0$, podemos fazer bem melhor tirando proveito do facto de termos um padrão que se repete na Rede de Bayes. Notem que quase todos os nós são idênticos, tanto em termos do domínio como da cpt, só $X_0$ se distingue.

Para tirar proveito dessa repetição, em vez de gerarmos uma Rede de Bayes formada por tantos nós quantos as variáveis $X_0$, $X_1$,$X_2$, ... $X_{Max}$, geramos apenas uma rede formada por dois nós: $X_t$, com a cpt correspondente à distribuição da posição corrente e $X_{t+1}$ com a mesma cpt que indica a distribuição de probabilidades das posições seguintes dadas as correntes. Para simular o tempo, teremos então de alimentar a Rede de Bayes com a distribuição corrente e inferir a distribuição de probabilidade no tempo seguinte usando a inferência da Rede de Bayes.

Exemplifiquemos o processo.

Geramos a Rede de Bayes padrão com dois nós (ver figura em cima):

    1. um correspondente à variável $X_t$ com domínio contendo todas as células do espaço. A distribuição inicial vai ser usada para preencher a cpt de $X_t$. Essa distribuição inicial é definida através da função `init_prob`.
    2. um segundo correspondente a $X_{t+1} e a cpt é preenchida com base na dinâmica do movimento expressa na função `go`.

Entremos no processo que se repete.

$P(X_{0})$ é conhecida porque corresponde à distribuição inicial.

Se quisermos inferir $P(X_1)$, afectamos a cpt de $P(X_t)$ a $P(X_0)$ e inferimos $P(X_{t+1})$ que é igual então a $P(X_1)$.

Se quisermos inferir $P(X_2)$, afectamos $P(X_1)$ à cpt da Rede de Bayes Padrão, que acabámos de inferir, e depois inferimos $P(X_{t+1})$ que é precisamente $P(X_2)$.

Assim, em geral, se quisermos inferir a distribuição marginal $P(X_n)$ teremos de repetir este processo um número n de vezes.

Deste modo geramos sempre apenas uma única Rede de Bayes formada por 2 nós e a inferência é também mais simples, embora tenha que ser repetida n vezes e o valor da cpt de $P(X_t)$ é no passo seguinte instanciada pelo valor acabado de inferir no passo imediatamente anterior. A cpt do nó $X_{t+1}$ nunca muda.

                                                       **
                                                       
O objectivo é construírem uma função `fantasma_repeat(distIni, max, movimentos, donut=False)`, que recebe como argumentos a distribuição inicial, o instante máximo da simulação e o modelo de movimento (representado nos dois argumentos: movimentos e donut), e devolve a distribuição marginal $P(X_{max})$.

**Nota:** para testar esta função, teremos de fazer uso das funções `initDist` e `go`, que estão no ficheiro anexo funcoes_fantasma.py e que foram encomendadas no mini-projecto I.

**Importante:** Para que não haja conflito entre a representação bidimensional do espaço e a forma como se representa as tabelas de probabilidades condicionais na classe `RedeBayes`, é necessário representar as células do espaço como `strings`. Por exemplo, a célula $1$ será $'1'$ e a célula $(3,3)$ será $'(3,3)'$.

Não têm de se preocupar com a programação defensiva. Os testes cumprem com os inputs pedidos.

### Testes da Função 2

#### Teste 1

```python
ini = initDist((3,3),[(1,1)])
f=fantasma_repeat(ini,5,['E','.'])
printSorted(f.prob)

Output:
{'(1, 1)': 0.03125, '(1, 2)': 0.15625, '(1, 3)': 0.8125, '(2, 1)': 0.0, '(2, 2)': 0.0, '(2, 3)': 0.0, '(3, 1)': 0.0, '(3, 2)': 0.0, '(3, 3)': 0.0}    
```

#### Teste 2

```python
ini = initDist(20,[1])
f=fantasma_repeat(ini,1000,['O','E','.'],True)
printSortedRound(f.prob,5)

Output:
{'1': 0.05, '10': 0.05, '11': 0.05, '12': 0.05, '13': 0.05, '14': 0.05, '15': 0.05, '16': 0.05, '17': 0.05, '18': 0.05, '19': 0.05, '2': 0.05, '20': 0.05, '3': 0.05, '4': 0.05, '5': 0.05, '6': 0.05, '7': 0.05, '8': 0.05, '9': 0.05}
```

#### Teste 3

```python
ini = initDist((10,10),[(1,1)])
f=fantasma_repeat(ini,50,['E','S','.'])
v=list(f.prob.values())
print('{0:.10f}'.format(max(v)))

Output:
0.9899339579
```

#### Teste 4

```python
lim=20
ini = initDist(lim,list(range(1,lim+1)))
f=fantasma_repeat(ini,10,['E'],True)
printSorted(f.prob)

Output:
{'1': 0.05, '10': 0.05, '11': 0.05, '12': 0.05, '13': 0.05, '14': 0.05, '15': 0.05, '16': 0.05, '17': 0.05, '18': 0.05, '19': 0.05, '2': 0.05, '20': 0.05, '3': 0.05, '4': 0.05, '5': 0.05, '6': 0.05, '7': 0.05, '8': 0.05, '9': 0.05}
```

#### Teste 5

```python
ini = initDist(50,[1])
f=fantasma_repeat(ini,60,{'E':0.9,'.':0.1},False)
v=list(f.prob.values())
print('{0:.10f}'.format(max(v)))

Output:
0.9854153148
```

<img src="files/imagens/paredes-portas-abertas-fechadas.PNG" alt="Drawing" style="width: 500px;"/>

## Função 3
Suponhamos agora que o fantasma vive apenas num mundo de 1D ladeado a norte por paredes, portas fechadas e portas abertas, mas que não consegue aceder ao estado, i.e., à sua posição. Ele continua a tentar perceber onde está, e não podendo aceder à sua posição concreta, pode sensoriar em seu redor e detectar se está perante uma parede, uma porta aberta ou uma porta fechada. O seu sensor se ficar vermelho ('v') indica em geral parede, laranja ('l') indica em geral porta fechada e amarelo ('a') indica em geral uma porta aberta. É um sensor naturalmente ruidoso, mas o fantasma conhece o seu modelo, i.e. sabe que por exemplo 10% das vezes que está perante uma parede, o sensor pode enganar-se e indicar uma porta fechada e também se pode enganar 5% das vezes, indicando uma porta aberta.  O fantasma conhece também a dinâmica do seu próprio movimento, que depende apenas da sua posição imediatamente anterior, definida através da função nossa conhecida `go` para além da distribuição da posição inicial, no instante T0, definida através da função `initDist`.

Um aspecto muito importante é que o vento exerce a sua força em cada instante mas o fantasma pode fazer uso do seu sensor e utilizá-lo para se localizar apenas de 2 em 2 tics, por exemplo, ou de 3 em 3 tics ou até tic a tic. O `periodo_sensor` quando tem o valor de 1 indica que o sensor é usado em cada instante, 2 indicará que é feita uma leitura de 2 em 2 tics, 10 indicará que o sensor é usado de 10 em 10 instantes, etc.

O objectivo do fantasma, que executa o seu movimento em cada instante é calcular a sua posição após ter recebido um conjunto de valores do sensor: por exemplo, imaginem que o sensor tem um período de 0 e que o fantasma se move num espaço donut com 10 células, ilustrado na figura em cima. Ele pretende conhecer a sua posição mais provável após 10 tiques, não tendo nenhuma ideia de onde partiu, e que se move para leste 80% das vezes, ficando na mesma célula as outras 20%. Ele fez 11 leituras ao sensor, uma no início e logo a seguir a cada movimento, e recebeu os seguintes valores, por esta ordem: 

```python
    v v v v l a a l l l l
```

Notem que o primeiro valor do sensor é sempre obtido na posição inicial X0.

O problema, com período 1, é modelizado através da seguinte Rede de Bayes:

<img src="files/imagens/cadeia-fantasma-sensores-velocidade1.PNG" alt="Drawing" style="width: 500px;"/>

Se o período for de 3, o padrão da rede que se repete será o seguinte:

<img src="files/imagens/padrao-fantasma-sensores-velocidade3.PNG" alt="Drawing" style="width: 500px;"/>

**Estratégia**: Como queremos em princípio poder simular o movimento do robô e respectiva localização durante um período que pode ser longo, não poderemos construír uma rede única com $t_{max}$ variáveis para a posição e $t_{max}/p$ para os sensores, sendo $p$ o período do sensor. A inferência demoraria uma eternidade. Teremos que optar por uma estratégia semelhante à da pergunta 2 deste quizz. Arranjar um elemento mínimo que se repete, ou até fazê-lo recursivamente para o caso de um período do sensor > 1, e fazer em cada passo a inferência cujo resultado passa a ser a probabilidade apriori no passo seguinte. 
 

#### Objectivo

A função `fantasma_track(initDist,landmarks, sensor, periodo, leituras, movimentos, donut=False,verbose=False)`, que recebe como argumentos a distribuição inicial, a indicação onde estão as paredes, portas aberta e fechadas (landmarks), o modelo do sensor e o seu periodo, a lista com as leituras ordenadas dos sensores, o modelo de movimento (representado nos dois argumentos movimentos, e o donut), bem como um boolean a indicar se queremos o modo verbose ou não, e devolve uma instância da classe `ProbDist` que indica a distribuição de probabilidade condicional da variável correspondente à posição final, dados os valores obtidos dos sensores, e depois de ter executado o seu movimento em cada instante. Por exemplo, se o fantasma tiver um período de 2 no seu sensor, e usar o seu sensor 3 vezes, então pretende-se conhecer a sua posição no instante final, o instante 5.

Nota: para testar esta função, teremos de fazer uso das funções `go` e da `initDist`.

A indicação do mapa em termos de portas, abertas e fechadas e de paredes é dada por uma lista ordenada da esquerda para a direita, em que: 

    'p' representa parede
    'pa' representa porta aberta
    'pf' representa porta fechada

Por exemplo, o cenário da figura no topo é representado por esta lista:

```python
['pa','p','p','p','pa','pf''p','p','pf','pa']
```

O modelo dos sensores é dado sob a forma de um dicionário. Eis um exemplo:
``` python
sensor={'p': {'v':0.8,'l':0.15,'a':0.05}, 
        'pf': {'v':0.05,'l':0.8,'a':0.15}, 
        'pa': {'v':0.02,'l':0.13,'a':0.85}}
```

Não têm de se preocupar com a programação defensiva. Os testes cumprem com os inputs pedidos.

### Testes da Função 3

#### Teste 1
    # O espaço é o da figura.
    # Pode estar em qualquer das células com igual probabilidade.
    # Os sensores são exactos: vermelho para a parede, laranja para a porta fechada e amarelo para a porta aberta
    # O período é de 1.
    # Sensores: laranja, vermelho, vermelho.
    # O movimento é só para leste, sem erro.
    # donut = True
    # verbose: False
    
Output: a distribuição de probabilidade da sua localização (X2) após 3 leituras dos sensores.

```python
landmarks=['pa','p','p','p','pa','pf','p','p','pf','pa']
sensor={'p': {'v':1.0,'l':0,'a':0}, 'pf': {'v':0,'l':1.0,'a':0}, 'pa': {'v':0,'l':0,'a':1.0}}
valores_sensores=['l','v','v']
ini=initDist(10,list(range(1,11)))
periodo=1
movimentos=['E']
don=True
out=fantasma_track(ini,landmarks, sensor, periodo, valores_sensores, movimentos, don)
printSorted(out.prob)

Output:
{'1': 0.0, '10': 0.0, '2': 0.0, '3': 0.0, '4': 0.0, '5': 0.0, '6': 0.0, '7': 0.0, '8': 1.0, '9': 0.0}
```

#### Teste 2
Tudo igual ao teste 1 mas com o modo verbose ligado

```Python
ini=initDist(10,list(range(1,11)))
landmarks=['pa','p','p','p','pa','pf','p','p','pf','pa']
sensor={'p': {'v':1.0,'l':0,'a':0}, 'pf': {'v':0,'l':1.0,'a':0}, 'pa': {'v':0,'l':0,'a':1.0}}
periodo=1
valores_sensores=['l','v','v']
movimentos=['E']
out=fantasma_track(ini,landmarks, sensor, periodo, valores_sensores, movimentos, True,True)
printSorted(out.prob)

Output:
X0: {'1': 0.1, '10': 0.1, '2': 0.1, '3': 0.1, '4': 0.1, '5': 0.1, '6': 0.1, '7': 0.1, '8': 0.1, '9': 0.1}
Sensor em t=0: l
X0: {'1': 0.0, '10': 0.0, '2': 0.0, '3': 0.0, '4': 0.0, '5': 0.0, '6': 0.5, '7': 0.0, '8': 0.0, '9': 0.5}
Sensor em t=1: v
X1: {'1': 0.0, '10': 0.0, '2': 0.0, '3': 0.0, '4': 0.0, '5': 0.0, '6': 0.0, '7': 1.0, '8': 0.0, '9': 0.0}
Sensor em t=2: v
X2: {'1': 0.0, '10': 0.0, '2': 0.0, '3': 0.0, '4': 0.0, '5': 0.0, '6': 0.0, '7': 0.0, '8': 1.0, '9': 0.0}
{'1': 0.0, '10': 0.0, '2': 0.0, '3': 0.0, '4': 0.0, '5': 0.0, '6': 0.0, '7': 0.0, '8': 1.0, '9': 0.0}
```


#### Teste 3
Tudo igual ao teste 1, excepto que agora vai sempre para oeste, sem falhar, mas com um período de leitura dos sensores de 4.

```python
ini=initDist(10,list(range(1,11)))
landmarks=['pa','p','p','p','pa','pf','p','p','pf','pa']
sensor={'p': {'v':1.0,'l':0,'a':0}, 'pf': {'v':0,'l':1.0,'a':0}, 'pa': {'v':0,'l':0,'a':1.0}}
periodo=4
valores_sensores=['l','v','v']
movimentos=['O']
don=True
out=fantasma_track(ini,landmarks, sensor, periodo, valores_sensores, movimentos, don)
printSorted(out.prob)

Output:
{'1': 0.0, '10': 0.0, '2': 0.0, '3': 0.0, '4': 0.0, '5': 0.0, '6': 0.0, '7': 0.0, '8': 1.0, '9': 0.0}
```

#### Teste 4
Vamos agora ter uma maior leitura dos sensores, um período de 6 e os sensores + movimento ambos incertos

```python
landmarks=['pa','p','p','p','pa','pf','p','p','pf','pa','pa','p','p','p','pa','pf','p','p','pf','pa']
ini=initDist(len(landmarks),list(range(1,len(landmarks)+1)))
sensor={'p': {'v':0.8,'l':0.15,'a':0.05}, 
        'pf': {'v':0.05,'l':0.8,'a':0.15}, 
        'pa': {'v':0.02,'l':0.13,'a':0.85}}
movimentos={'E':0.9, '.': 0.1}
valores_sensores=['l','v','v','v','a','l','v','v','v','v','l','v','v','v','v','l','v','v','v','a','l','v']
ps=6
out=fantasma_track(ini,landmarks, sensor, ps, valores_sensores, movimentos, True)
v=list(out.prob.values())
print('{0:.5f}'.format(max(v)))

Output:
0.44042
```

## Como submeter o projeto

O grupo de alunos deve implementar as 3 funções pedidas (com cotações, por ordem: 0.7+0.7+0.6) e testá-las o melhor possível, após o que **um único aluno do grupo** deve responder ao *quizz* **Projeto2** que está na página da cadeira, introduzindo aí o código das funções.

Esse *quizz* é constituído por 3 perguntas, que correspondem a cada uma das 3 funções pedidas. Cada função é avaliada com um conjunto de testes visíveis e mais alguns testes escondidos.

A última submissão é a que será considerada. Se um grupo fizer várias submissões, elas devem ser todas submetidas pelo aluno do grupo que fez a primeira submissão. Mas, se houverem submissoes de diferentes elementos do mesmo grupo, será considerada a maior nota.